In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('cian.csv', low_memory=False)

In [3]:
df.columns = [i.replace('cian.', '') for i in df.columns]

In [4]:
df

,cid,type,property_type,category,url,creation_date,last_update_date,location_country,location_locality_name,location_sub_locality_name,location_address,location_latitude,location_longitude,metro_name,metro_time_on_foot,sales_agent_name,sales_agent_phone,sales_agent_category,sales_agent_agency,price_value,price_currency,price_deal_status,price_mortgage,area_value,area_unit,living_space_value,living_space_unit,kitchen_space_value,kitchen_space_unit,image,description,rooms,floor,floors_total,commercial_type,location_direction,location_distance,location_region,location_district,vas,cian_building_id,building_name,lot_area_value,lot_area_unit,metro_time_on_transport,garage_type,price_period,price_agent_fee,price_rent_pledge,renovation,room_space_value,room_space_unit,new_flat,rooms_offered,cian_house_id,studio,building_state,built_year,ready_quarter,views,etl_date
0,10062356,аренда,жилая,flat,https://www.cian.ru/rent/flat/10062356/,2013-09-02 18:56:52.0,2022-04-29 06:38:27.0,Россия,Москва,"СВАО, р-н Бутырский","Бутырская улица, 2/18",55.794416,37.586509,Савеловская; Менделеевская; Дмитровская,3; 20,64320,79096867429,agency,64320,3990.0,RUB,NaN,NaN,64.0,кв. м,55.0,кв. м,7.0,кв. м,http://pmt.pw/c/images/90/283/kvartira-moskva-...,Заселение круглосуточно. Стоимость зависит от ...,2.0,11.0,15.0,NaN,NaN,NaN,NaN,NaN,pro,NaN,NaN,NaN,NaN,2,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2577,2022-06-01
1,10340177,аренда,жилая,flat,https://www.cian.ru/rent/flat/10340177/,2013-10-27 11:44:28.0,2022-04-29 07:18:43.0,Россия,Москва,"ЮВАО, р-н Марьино","Новочеркасский бульвар, 45",55.648910,37.738108,Марьино,4,Брусника Холдинг,79660508062,agency,Брусника Холдинг,2300.0,RUB,NaN,NaN,31.0,кв. м,23.0,кв. м,7.0,кв. м,http://pmt.pw/c/images/80/345/611/posutochnaja...,"Посуточно. Квартира стандарт-класса, в шаговой...",1.0,1.0,12.0,NaN,NaN,NaN,NaN,NaN,pro,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5371,2022-06-01
2,10340685,аренда,жилая,flat,https://www.cian.ru/rent/flat/10340685/,2013-10-27 14:19:39.0,2022-04-22 01:35:21.0,Россия,Москва,"ЦАО, р-н Арбат","переулок Калошин, 6С1",55.748321,37.592482,Арбатская; Смоленская,7; 8,Брусника Холдинг,79660508062,agency,Брусника Холдинг,3000.0,RUB,NaN,NaN,32.0,кв. м,18.0,кв. м,7.0,кв. м,http://pmt.pw/c/images/13/345/611/posutochnaja...,"Квартира бизнес-класса, в шаговой доступности ...",1.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN,pro,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6621,2022-06-01
3,10342702,аренда,жилая,flat,https://www.cian.ru/rent/flat/10342702/,2013-10-28 09:07:06.0,2022-04-23 03:25:05.0,Россия,Москва,"САО, р-н Сокол","Песчаная улица, 8",55.802609,37.514473,Сокол,3,Брусника Холдинг,79660508062,agency,Брусника Холдинг,2500.0,RUB,NaN,NaN,38.0,кв. м,20.0,кв. м,10.0,кв. м,http://pmt.pw/c/images/52/657/811/posutochnaja...,"Квартира стандарт-класса, в шаговой доступност...",1.0,7.0,12.0,NaN,NaN,NaN,NaN,NaN,"pro,auction",NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3744,2022-06-01
4,10404971,аренда,жилая,flat,https://www.cian.ru/rent/flat/10404971/,2013-11-08 19:33:22.0,2022-05-05 02:20:48.0,Россия,Москва,"ЦАО, р-н Арбат","Новинский бульвар, 1/2",55.750722,37.582412,Смоленская,3,BALMONT APARTMENTS,79660504516,agency,BALMONT APARTMENTS,5500.0,RUB,NaN,NaN,38.0,кв. м,30.0,кв. м,8.0,кв. м,http://pmt.pw/c/images/7/767/348/kvartira-mosk...,"Бальмонт Апартаменты.Уютная квартира - студия,...",1.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,pro,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,896,2022-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238150,9198334,продажа,жилая,townhouse,https://www.cian.ru/sale/suburban/9198334/,2015-10-13 09:18:01.0,2021-12-13 23:21:26.0,Россия,"Москва, Сосенское поселение, д.

In [27]:
df.set_index('cid', inplace=True)
columns_to_drop = ['url', 'location_country', 'sales_agent_phone', 'price_currency', 'image',
                   'vas', 'cian_building_id', 'garage_type', 'price_rent_pledge', 'renovation', 
                   'views', 'etl_date', 'location_district', 'property_type', 'price_mortgage',
                   'price_deal_status', 'sales_agent_name', 'sales_agent_agency',
                   'lot_area_value', 'lot_area_unit', 'room_space_value', 'room_space_unit',
                   'kitchen_space_unit', 'new_flat', 'rooms_offered', 'cian_house_id', 'building_state',
                   'built_year', 'ready_quarter']
df.drop(columns_to_drop, axis=1, inplace=True)

In [28]:
df = df[~df['area_unit'].isnull()].copy()
df.drop('area_unit', axis=1, inplace=True)

df = df[df['commercial_type'].isnull()].copy()
df.drop('commercial_type', axis=1, inplace=True)

df = df[df['location_region'].isnull()].copy()
df.drop('location_region', axis=1, inplace=True)

df = df[df['price_period'] != 'day']
df.drop('price_period', axis=1, inplace=True)

df = df[df['type']=='аренда']
df.drop('type', axis=1, inplace=True)

df = df[df['category']=='flat']
df.drop('category', axis=1, inplace=True)

In [29]:
df.drop(['location_locality_name', 'creation_date', 'last_update_date', 'building_name'], axis=1, inplace=True)

In [30]:
def cut_sub_locality(x):
    if 'ЮЗАО,' in x:
        return 'ЮЗАО'
    elif 'СЗАО,' in x:
        return 'СЗАО'
    elif 'СВАО,' in x:
        return 'СВАО'
    elif 'ЮВАО' in x:
        return 'ЮВАО'
    elif 'ЗАО,' in x:
        return 'ЗАО'
    elif 'ЦАО,' in x:
        return 'ЦАО'
    elif 'САО,' in x:
        return 'САО'
    elif 'ЮАО,' in x:
        return 'ЮАО'
    elif 'ВАО,' in x:
        return 'ВАО'
    elif 'НАО (Ново' in x:
        return 'НАО (Новомосковский)'
    elif 'ЗелАО,' in x:
        return 'ЗелАО'
    elif 'ТАО (Троицкий)' in x:
        return 'ТАО (Троицкий)'
    else:
        return x

In [31]:
df['location_sub_locality_name'].value_counts()

НАО (Новомосковский)            3875
ЦАО, р-н Пресненский            3046
ЦАО, р-н Тверской               1339
ЦАО, р-н Хамовники              1210
САО, р-н Хорошевский            1073
                                ... 
ЮАО, р-н Чертаново Северно��       1
ВАО, р-н Новоги��еево              1
ЮЗАО, р-н Ломоносо��ский           1
ЮВАО, р-н Нижегородс��ий           1
ЮВАО, ��-н Рязанский               1
Name: location_sub_locality_name, Length: 157, dtype: int64

In [32]:
df.location_sub_locality_name.map(lambda x: x[:3]).value_counts()

ЦАО    10476
ЗАО     7130
САО     6464
ЮЗА     5149
СВА     5124
ЮАО     5060
ЮВА     5000
ВАО     4465
НАО     4175
СЗА     3661
Зел      425
ТАО      251
Name: location_sub_locality_name, dtype: int64

In [33]:
df['location_sub_locality_name'] = df['location_sub_locality_name'].fillna('undef').map(cut_sub_locality)

In [34]:
df

,location_sub_locality_name,location_address,location_latitude,location_longitude,metro_name,metro_time_on_foot,sales_agent_category,price_value,area_value,living_space_value,living_space_unit,kitchen_space_value,description,rooms,floor,floors_total,location_direction,location_distance,metro_time_on_transport,price_agent_fee,studio
cid,,,,,,,,,,,,,,,,,,,,,
10712873,ЦАО,"улица Гашека, 11",55.769707,37.591117,Маяковская; Белорусская; Тверская,6; 10; 13,agency,100000.0,45.0,27.0,кв. м,8.0,"ID 105 Сдается от 1 мес!Отчетные документы, ре...",2.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN
12833884,ЦАО,"улица Остоженка, 25",55.739554,37.597414,Кропоткинская; Парк Культуры,7; 7,agency,500000.0,150.0,100.0,кв. м,50.0,Агенту Бонус 50/50 ЖК Опера Хаус. В аренду пр...,3.0,2.0,7.0,NaN,NaN,NaN,NaN,NaN
145766778,ЦАО,"улица Шаболовка, 23к4",55.722162,37.613090,Шаболовская,8,agency,140000.0,114.0,64.0,кв. м,20.0,"ЖК Созвездие капитал-1, Предлагается в аренду ...",3.0,3.0,20.0,NaN,NaN,NaN,NaN,NaN
148813955,ЮВАО,"Окская улица, 48/2",55.717655,37.779422,Окская,5,owner,43000.0,35.0,20.0,кв. м,10.0,Сдаётся 1-ая квартира на длительный срок от со...,1.0,11.0,12.0,NaN,NaN,NaN,NaN,NaN
148978886,САО,"Часовая улица, 23к1",55.809975,37.521830,Сокол; Красный Балтиец; Аэропорт,7; 7; 18,agency,300000.0,167.0,95.0,кв. м,20.0,Агенту Бонус! Предлагается в аренду современн...,4.0,16.0,23.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274061445,ЮЗАО,"улица Ферсмана, 3К1",55.695372,37.574148,Академическая,11,agency,80000.0,83.0,63.0,кв. м,10.0,Сдаётся четырехкомнатная квартира на улице Фер...,4.0,2.0,8.0,NaN,NaN,NaN,50.0,NaN
274064448,САО,"улица Адмирала Макарова, 6Бк1",55.835766,37.491862,Водный стадион; Балтийская; Войковская,7; 15,agency,80000.0,41.0,NaN,NaN,7.0,Предлагается уютная двухкомнатная квартира в к...,2.0,9.0,25.0,NaN,NaN,5,50.0,NaN
274065318,ЗАО,"Производственная улица, 2к1",55.649500,37.387846,Солнцево,2,agency,80000.0,100.0,58.0,кв. м,10.0,"Впервые сдаётся 4-х комнатная квартира, общая ...",4.0,2.0,22.0,Боровское,4.0,NaN,40.0,NaN


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57380 entries, 10712873 to 9761071
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   location_sub_locality_name  57380 non-null  object 
 1   location_address            57299 non-null  object 
 2   location_latitude           57380 non-null  float64
 3   location_longitude          57380 non-null  float64
 4   metro_name                  56213 non-null  object 
 5   metro_time_on_foot          49010 non-null  object 
 6   sales_agent_category        57380 non-null  object 
 7   price_value                 57380 non-null  float64
 8   area_value                  57380 non-null  float64
 9   living_space_value          44990 non-null  float64
 10  living_space_unit           44990 non-null  object 
 11  kitchen_space_value         47489 non-null  float64
 12  description                 57380 non-null  object 
 13  rooms                 

In [36]:
df['sales_agent_category'].value_counts()

agency    46826
owner     10554
Name: sales_agent_category, dtype: int64

In [37]:
df = df[df['location_sub_locality_name']!='ЗелАО']
df = df[df['location_sub_locality_name']!='ТАО (Троицкий)']

In [38]:
df['location_sub_locality_name'].value_counts()

ЦАО                     10476
ЗАО                      7130
САО                      6464
ЮЗАО                     5149
СВАО                     5124
ЮАО                      5060
ЮВАО                     5000
ВАО                      4465
НАО (Новомосковский)     4175
СЗАО                     3661
Name: location_sub_locality_name, dtype: int64

In [39]:
df.to_csv('cian_short.csv')